In [10]:
# 忽略各模块的警告信息
import warnings
warnings.filterwarnings("ignore")

In [11]:
import akshare as ak

szse_all_stock = ak.stock_sz_a_spot_em().代码.to_list()
vt_symbols = [f"{stock}.SZSE" for stock in szse_all_stock]

# print(vt_symbols)


  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
# 加载所需使用的模块
from datetime import datetime

from vnpy.trader.datafeed import get_datafeed
from vnpy.trader.database import get_database, DB_TZ
from vnpy.trader.constant import Interval
from vnpy.trader.object import BarData, HistoryRequest
from vnpy.trader.utility import extract_vt_symbol
from vnpy.trader.setting import SETTINGS
from akshare import stock_feature as sf

# 配置数据服务
SETTINGS["datafeed.name"] = "akshare"            # 可以根据自己的需求选择数据服务：rqdata/xt/wind等
SETTINGS["datafeed.username"] = "license"       # RQData的用户名统一为“license”这个字符串
SETTINGS["datafeed.password"] = "123456"        # 这里需要替换为你购买或者申请试用的RQData数据license

# 创建对象实例
datafeed = get_datafeed()

# 要下载数据的起止时间
start = datetime(2019, 1, 1, tzinfo=DB_TZ)
end = datetime(2025, 7, 4, tzinfo=DB_TZ)

In [ ]:
# 遍历列表执行下载
count = 0
vt_symbols = vt_symbols[1020:]
size = len(vt_symbols)
for vt_symbol in vt_symbols:
    # 拆分合约代码和交易所
    symbol, exchange = extract_vt_symbol(vt_symbol)

    # 创建历史数据请求对象
    req: HistoryRequest = HistoryRequest(
        symbol=symbol,
        exchange=exchange,
        start=start,
        end=end,
        interval=Interval.DAILY
    )

    # 从数据服务下载数据
    retry = 3
    bars: list[BarData] = []
    while retry > 0:
        try:
            bars = datafeed.query_bar_history(req)
            break
        except Exception as e:
            print(f"连接数据服务异常：{vt_symbol}，错误信息：{e}")
            retry -= 1

    if len(bars) > 0:
        # print(bars)
        database.save_bar_data(bars)
        count += 1
        print(f"[{count}/{size}] 下载数据成功：{vt_symbol}，总数据量：{len(bars)}")
    else:
        print(f"下载数据失败：{vt_symbol}")

    

[1/2022] 下载数据成功：001373.SZSE，总数据量：507
[2/2022] 下载数据成功：002412.SZSE，总数据量：1577
[3/2022] 下载数据成功：300058.SZSE，总数据量：1577
[4/2022] 下载数据成功：300363.SZSE，总数据量：1577
[5/2022] 下载数据成功：000410.SZSE，总数据量：1560
[6/2022] 下载数据成功：301048.SZSE，总数据量：938
[7/2022] 下载数据成功：002749.SZSE，总数据量：1577
[8/2022] 下载数据成功：002785.SZSE，总数据量：1577
[9/2022] 下载数据成功：002033.SZSE，总数据量：1577
[10/2022] 下载数据成功：002830.SZSE，总数据量：1577
[11/2022] 下载数据成功：002461.SZSE，总数据量：1577
[12/2022] 下载数据成功：300576.SZSE，总数据量：1567
[13/2022] 下载数据成功：002777.SZSE，总数据量：1577
[14/2022] 下载数据成功：300677.SZSE，总数据量：1577
[15/2022] 下载数据成功：000065.SZSE，总数据量：1571
[16/2022] 下载数据成功：000638.SZSE，总数据量：1574
[17/2022] 下载数据成功：300627.SZSE，总数据量：1577
[18/2022] 下载数据成功：300755.SZSE，总数据量：1558
[19/2022] 下载数据成功：002709.SZSE，总数据量：1577
[20/2022] 下载数据成功：300629.SZSE，总数据量：1566
[21/2022] 下载数据成功：002715.SZSE，总数据量：1567


In [3]:
from datetime import datetime

from vnpy.trader.datafeed import get_datafeed
from vnpy.trader.database import get_database, DB_TZ
from vnpy.trader.constant import Interval
from vnpy.trader.object import BarData, HistoryRequest
from vnpy.trader.utility import extract_vt_symbol
from vnpy.trader.setting import SETTINGS
from akshare import stock_feature as sf
# data = database.load_bar_data('000001', Exchange.SZSE, Interval.DAILY, start, end)

# 配置数据库
SETTINGS["database.name"] = "mysql"              # 可以根据自己的需求选择数据库，这里使用的是TDengine
SETTINGS["database.database"] = "SZSE"
SETTINGS["database.host"] = "127.0.0.1"
SETTINGS["database.port"] = 3306
SETTINGS["database.user"] = "root"
SETTINGS["database.password"] = "123456"

database = get_database()

all_symbols = database.get_all_symbols(Exchange.SZSE,
                                    interval=Interval.DAILY,
                                    start=datetime(2023, 7, 30),
                                    end=datetime(2025, 7, 30))
print(f"所有合约数量：{len(all_symbols)}")


所有合约数量：1059


In [5]:
from vnpy.trader.optimize import OptimizationSetting
from vnpy_ctastrategy.backtesting import BacktestingEngine
from vnpy_ctastrategy.strategies.atr_rsi_strategy import AtrRsiStrategy
from strategies.multi_signal_strategy import MultiSignalStrategy
from strategies.buy_and_hold_strategy import BuyAndHoldStrategy

engine = BacktestingEngine()
goods_symbols = []

# end_pos = len(all_symbols)
end_pos = 10
symbols_candidates = ["000785"]
for symbol in symbols_candidates:
    engine.set_parameters(
        vt_symbol=symbol + ".SZSE",
        interval="d",
        start=datetime(2025, 1, 30),
        end=datetime(2025, 7, 30),
        rate=0.3/10000,
        slippage=0.2,
        size=300,
        pricetick=0.2,
        capital=1_000_000,
    )
    engine.add_strategy(BuyAndHoldStrategy, {})

    engine.load_data()
    engine.run_backtesting()
    df = engine.calculate_result()
    print(f'df: {df}')
    res = engine.calculate_statistics()
    print(f"策略回测成功：{symbol}，总净利润：{res['total_net_pnl']}")
    if res["total_net_pnl"] > 0:
        goods_symbols.append({"symbol": symbol, "total_net_pnl": res["total_net_pnl"]})
        # print(f"策略回测成功：{symbol}，总净利润：{res['total_net_pnl']}")

print(goods_symbols)

    # df.
    # engine.show_chart()

2025-07-11 15:29:28.345274	开始加载历史数据
2025-07-11 15:29:28.345938	加载进度：# [0%]
2025-07-11 15:29:28.346455	加载进度：# [10%]
2025-07-11 15:29:28.346524	加载进度：## [20%]
2025-07-11 15:29:28.346534	加载进度：### [30%]
2025-07-11 15:29:28.346542	加载进度：#### [40%]
2025-07-11 15:29:28.346550	加载进度：##### [50%]
2025-07-11 15:29:28.346558	加载进度：###### [60%]
2025-07-11 15:29:28.346567	加载进度：####### [70%]
2025-07-11 15:29:28.346574	加载进度：######## [80%]
2025-07-11 15:29:28.346582	加载进度：######### [90%]
2025-07-11 15:29:28.346627	加载进度：########## [99%]
2025-07-11 15:29:28.346656	历史数据加载完成，数据量：97
2025-07-11 15:29:28.346913	策略初始化完成
2025-07-11 15:29:28.346961	开始回放历史数据
2025-07-11 15:29:28.354107	回放进度：= [0%]
2025-07-11 15:29:28.354165	回放进度：== [10%]
2025-07-11 15:29:28.354201	回放进度：=== [20%]
2025-07-11 15:29:28.354235	回放进度：==== [30%]
2025-07-11 15:29:28.354271	回放进度：===== [40%]
2025-07-11 15:29:28.354304	回放进度：====== [50%]
2025-07-11 15:29:28.354337	回放进度：======= [60%]
2025-07-11 15:29:28.354390	回放进度：======== [70%]
2025-07-11 15:29:28

In [ ]:
setting = OptimizationSetting()
setting.set_target("sharpe_ratio")
setting.add_parameter("atr_length", 25, 27, 1)
setting.add_parameter("atr_ma_length", 10, 30, 10)

engine.run_ga_optimization(setting)

In [ ]:
engine.run_bf_optimization(setting)